# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS). This will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 14/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [299]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import product

# **Layer Function & Class Definitions:**

Need to define the membership function to be used first:

In [300]:
# triangular membership function:
def triangular_mf(x, params):
    a, b, c = params
    y = np.zeros_like(x, dtype=float)  # initialize output array

    # define membership function rules using numpy indexing
    mask1 = (x > a) & (x < b)  # rising edge of the triangle
    mask2 = (x >= b) & (x < c)  # falling edge of the triangle

    y[mask1] = (x[mask1] - a) / (b - a)
    y[mask2] = (c - x[mask2]) / (c - b)

    return y

Now need to define the ANFIS class:

In [301]:
class ANFIS:
    # object constructor:
    def __init__(self, num_inputs, num_mfs, params = None):
        # need to instantiate the object:
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs
        self.memberships = {}
        self.consequents = {}

        # must first assign the antecedent parameters:
        # if custom:
        if params is not None:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    self.memberships[f'membership_{j+1}_{i+1}'] = params[j, i, :]
            print('params set to other than None')
        # if not custom, randomly initialize them:
        else:
            for j in range(self.num_inputs):
                for i in range(self.num_mfs):
                    a = np.random.uniform(low = -1.0, high = 1.0)
                    b = np.random.uniform(low = -1.0, high = 1.0)
                    c = np.random.uniform(low = -1.0, high = 1.0)

                    params = np.array([a, b, c])
                    self.memberships[f'membership_{j+1}_{i+1}'] = params

        # now must assign the consequent parameters:
        for rule_index in range(1, self.num_rules + 1):
            params = np.random.uniform(low = -1.0, high = 1.0, size = self.num_inputs + 1)
            self.consequents[f'rule_{rule_index}_params'] = params

        print('model created!')

    # this is a plotting function to verify that the membership functions are correct:
    def plot_membership_functions(self, max_values, mf_names = None):
        # if the user does not provide names:
        if mf_names is None:
            mf_names = [f'MF {i + 1}' for i in range(self.num_mfs)]

        # make sure that the number of names matches the number of membership functions:
        if len(mf_names) != self.num_mfs:
            raise ValueError(f'Expected {self.num_mfs} membership functions, but got {len(mf_names)}')

        # make sure that the number of provided max values matches the number of input functions:
        if len(max_values) != self.num_inputs:
            raise ValueError(f'Expected {self.num_inputs} max values, but got {len(max_values)}')
        
        # if matching, create a linspace based on the max values:
        input_ranges = {}
        for i in range(self.num_inputs):
            input_ranges[i] = np.linspace(0, max_values[i], 1000)

        # plot mfs for each input:
        for input_index in range(self.num_inputs):
            x_values = input_ranges[input_index]
            plt.figure(figsize=(12, 8))

            # plot each mfs for the selected input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{input_index + 1}_{i + 1}']
                y_values = [triangular_mf(x, params) for x in x_values]
                plt.plot(x_values, y_values, label = f'{mf_names[i]}')
        
                plt.title(f'Membership Functions for Input X{input_index + 1}')
                plt.xlabel('Input Value')
                plt.ylabel('Degree of Membership')
                plt.legend()
                plt.grid(True)
        plt.show()
    
    # this is the first layer within the anfis, the membership layer:
    def membership_layer(self, inputs):
        """
        inputs: np.ndarray of shape (batch_size, num_inputs)
        returns: dict with membership values for the entire batch

        """

        membership_values = {}
        batch_size = inputs.shape[0]
        # for every input j:
        for j in range(self.num_inputs):
            # for every membership function for that input:
            for i in range(self.num_mfs):
                params = self.memberships[f'membership_{j+1}_{i+1}']
                membership_values[f'membership_{j+1}_{i+1}'] = triangular_mf(inputs[:,j], params)
        
        return membership_values
    
    # this is the second layer within the anfis, the firing strength layer:
    def firing_strength_layer(self, membership_values):
        """
        membership_values: dict of membership values for the batch
        returns: np.ndarry of shape (batch_size, num_rules)

        """

        batch_size = next(iter(membership_values.values())).shape[0]  # Get batch size from membership values
        firing_strengths = np.ones((batch_size, self.num_rules))  # Initialize with ones for multiplication

        # Generate rule combinations
        rules = list(product(range(self.num_mfs), repeat=self.num_inputs))  # All rule combinations

        for rule_index, combination in enumerate(rules):
            for input_index, mf_index in enumerate(combination):
                key = f'membership_{input_index + 1}_{mf_index + 1}'
                firing_strengths[:, rule_index] *= membership_values[key]

        return firing_strengths
   
    # this is the third layer within the anfis, the normalization layer:
    def normalization_layer(self, firing_strengths):
        """
        firing_strengths: np.ndarray of shape (batch_size, num_rules)
        returns: np.ndarray of shape (batch_size, num_rules)
        """

        total_strength = np.sum(firing_strengths, axis = 1, keepdims = True)
        normalized_firing_strengths = firing_strengths / total_strength

        return normalized_firing_strengths
    
    # this is the fourth layer within the anfis, the rule consequent layer:
    def consequent_layer(self, normalized_firing_strengths, inputs):
        """
        normalized_firing_strengths: np.ndarray of shape (batch_size, num_rules)
        inputs: np.ndarray of shape (batch_size, num_inputs)
        returns: np.ndarray of shape (batch_size, num_rules)

        """
        batch_size = inputs.shape[0]
        consequents = np.zeros((batch_size, self.num_rules))

        for rule_index in range(1, self.num_rules + 1):
            params = self.consequents[f'rule_{rule_index}_params']
            # compute the consequent output for the entire batch
            consequents[:, rule_index - 1] = normalized_firing_strengths[:, rule_index - 1] * (
                np.dot(inputs, params[:-1]) + params[-1]
            )

        return consequents

    # this is the fifth layer within the anfis, the output layer:
    def output_layer(self, consequents):
        """
        consequents: np.ndarray of shape (batch_size, num_rules)
        returns: np.ndarray of shape (batch_size,)

        """
        return np.sum(consequents, axis=1)

    # all together, the forward pass through the network is given by:
    def forward_pass(self, inputs):
        # need to first pass inputs through the first layer to fuzzify them:
        fuzzified = self.membership_layer(inputs)

        # now we calculate the firing strength of each rule:
        firing_strengths = self.firing_strength_layer(fuzzified)

        # now we normalize these firing strengths:
        normalized_firing_strengths = self.normalization_layer(firing_strengths)

        # determine the rule consequents:
        consequents = self.consequent_layer(normalized_firing_strengths, inputs)

        # overall network output:
        output = self.output_layer(consequents)

        # return to user:
        return output
    

# **Importing the Training Data:**

With the model now defined, we can now load and split the data accordingly.

In [302]:
# load the training data:
data = pd.read_csv('V3_Data.csv')

# split into X and Y:
x_data = data.drop('Suitability', axis = 1)
y_data = data['Suitability']

# split into training, validation, and testing:
x_train, x_filler, y_train, y_filler = train_test_split(x_data, y_data, test_size = 0.2)
x_val, x_test, y_val,  y_test = train_test_split(x_filler, y_filler, test_size = 0.5)

Pre-batch the data:

In [303]:
batch_size = 32
num_batches = len(x_train) // batch_size

for batch_index in range(num_batches):
    # get the batch of data
    start = batch_index * batch_size
    end = (batch_index + 1) * batch_size
    x_batch = x_train[start:end]
    y_batch = y_train[start:end]

# **Using the Model:**

With the data now prepared, we can instantiate and utilize the model.

Define model parameters:

In [304]:
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

max_values = np.array([10, 25, 50])

Create the model:

In [305]:
model = ANFIS(num_inputs = 3, num_mfs = 3, params = params)
inputs = np.array([[2, 11, 21], [6, 21, 48]])

params set to other than None
model created!
